In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [2]:
zomato = pd.read_csv('zomato.csv', encoding='latin-1')

In [3]:
df = pd.DataFrame((zomato[['Cuisines', 'Rating text', 'Average Cost for two', 'Price range', 'Aggregate rating', 'Votes']]))

In [4]:
df.head(3)

,Cuisines,Rating text,Average Cost for two,Price range,Aggregate rating,Votes
0,"French, Japanese, Desserts",Excellent,1100,3,4.8,314
1,Japanese,Excellent,1200,3,4.5,591
2,"Seafood, Asian, Filipino, Indian",Very Good,4000,4,4.4,270


In [5]:
df.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [6]:
df.head(3)

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
0,"French, Japanese, Desserts",Excellent,1100,3,4.8,314
1,Japanese,Excellent,1200,3,4.5,591
2,"Seafood, Asian, Filipino, Indian",Very Good,4000,4,4.4,270


In [7]:
dfColunasParaDicionarizar = df[~df["cozinha"].str.contains(" ", na=False)]

In [8]:
ColunaCozinha = dfColunasParaDicionarizar['cozinha']

In [9]:
ColunaCozinha = ColunaCozinha.astype('str')

In [10]:
ColunaTaxaVotos = dfColunasParaDicionarizar['taxa_votos']

In [11]:
ColunaTaxaVotos = ColunaTaxaVotos.astype('str')

In [12]:
X = pd.concat([ColunaCozinha, ColunaTaxaVotos], axis=1)

In [13]:
labelencoder_dict = {}
onehotencoder_dict = {}

X_train = None
for i in range(0, X.shape[1]):
    label_encoder = preprocessing.LabelEncoder()
    labelencoder_dict[i] = label_encoder
    feature = label_encoder.fit_transform(X.iloc[:,i])
    #X_train.iloc[:,:]
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    onehotencoder_dict[i] = onehot_encoder
    if X_train is None:
          X_train = feature
    else:
          X_train = np.concatenate((X_train, feature), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

In [14]:
def getEncoded(test_data,labelencoder_dict,onehotencoder_dict):
    test_encoded_x = None
    for i in range(0,test_data.shape[1]):
        label_encoder =  labelencoder_dict[i]
        feature = label_encoder.transform(test_data.iloc[:,i])
        feature = feature.reshape(test_data.shape[0], 1)
        onehot_encoder = onehotencoder_dict[i]
        feature = onehot_encoder.transform(feature)
        if test_encoded_x is None:
              test_encoded_x = feature
        else:
              test_encoded_x = np.concatenate((test_encoded_x, feature), axis=1)
    return test_encoded_x

In [15]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
#X = df['cozinha']

In [16]:
dfProcessado = df[~df["cozinha"].str.contains(" ", na=False)]

In [17]:
dfProcessado.head(2)

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
1,Japanese,Excellent,1200,3,4.5,591
5,Chinese,Very Good,1000,3,4.4,336


In [18]:
dfProcessado.shape

(1679, 6)

In [19]:
x_treinamentoParte1 = X_train

In [20]:
treinamentoParte2 = pd.DataFrame((dfProcessado[['alcance_preco', 'classifi_agregada', 'votos']]))

In [21]:
treinamentoParte2.head(4)

,alcance_preco,classifi_agregada,votos
1,3,4.5,591
5,3,4.4,336
9,3,4.8,532
13,3,4.2,458


In [31]:
x_treinamentoParte2 = np.array(treinamentoParte2)

In [32]:
X_TRAIN = np.concatenate((x_treinamentoParte1, x_treinamentoParte2), axis=1)

In [33]:
X_TRAIN.shape

(1679, 82)

In [34]:
Y_TRAIN = np.array(dfProcessado.media_preco)

In [35]:
Y_TRAIN.shape

(1679,)

In [22]:
import xgboost as xgb
from sklearn import model_selection

In [37]:
xg_boost = xgb.XGBClassifier()

In [38]:
X_TRAIN.shape

(1679, 82)

In [39]:
Y_TRAIN.shape

(1679,)

In [41]:
xg_boost.fit(X_TRAIN[:100,:], Y_TRAIN[:100])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [43]:
print(xg_boost.score(X_TRAIN, Y_TRAIN))

#scores = model_selection.cross_val_score(xg_boost, X_TRAIN, Y_TRAIN, cv=5, scoring='accuracy')
#print(scores)
#print("Kfold on XGBClassifier: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std()))

0.08695652173913043


In [44]:
# Realizando uma predição

In [27]:
escolha = ['Japanese', 'Excellent', 4, 600, 4]

In [28]:
def escolhaParaVetorPredicao(escolhaLista):
    colunasCategoricasPredicao = [{'cozinha': escolhaLista[0], 'votos': escolhaLista[1]}]
    colunasCategoricasPredicaoDic = pd.DataFrame(colunasCategoricasPredicao)
    return colunasCategoricasPredicaoDic

In [29]:
#colunasCategoricasPredicao = [{'cozinha': escolha[0], 'votos': escolha[1]}]
#colunasCategoricasPredicaoDic = pd.DataFrame(colunasCategoricasPredicao)

In [30]:
dfescolhaParaVetorPredicao = escolhaParaVetorPredicao(escolha)

In [31]:
dicionarioCategoriasPredicaoParte1 = getEncoded(dfescolhaParaVetorPredicao,labelencoder_dict,onehotencoder_dict)

In [32]:
def concatenaParte1e2Dicionario(dicionarioCategoriasPredicaoParte1, escolha):
    PegandoParte2Escolha = np.asarray(escolha[2:])
    Parte2Escolha = PegandoParte2Escolha.reshape(1,3)
    escolha_para_predicao = np.concatenate((dicionarioCategoriasPredicaoParte1, Parte2Escolha), axis=1)
    
    return escolha_para_predicao

In [33]:
dfConcatenadoPraPredicao = concatenaParte1e2Dicionario(dicionarioCategoriasPredicaoParte1, escolha)

In [58]:
print("Média de preço para características buscadas: ", xg_boost.predict(dfConcatenadoPraPredicao))

Média de preço para características buscadas:  [170]


# Salvando o modelo do treinamento

In [23]:
import pickle

In [20]:
pickle.dump(xg_boost, open('modeloTreinadoMoveMe.sav', 'wb'))

NameError: name 'xg_boost' is not defined

# Abrindo o modelo para classificação

In [25]:
arquivo = open('modeloTreinadoMoveMe.sav', 'rb')

In [26]:
modelo_carregado = pickle.load(arquivo)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
modelo.predict(escolha_para_predicao)

# Realizando um predição com o modelo salvo

In [34]:
print("Média de preço predição carregada para características buscadas: ", modelo_carregado.predict(dfConcatenadoPraPredicao))

Média de preço predição carregada para características buscadas:  [230]
